# F1Ops Fleet Optimization

This notebook explores fleet allocation strategies for F1 team logistics using greedy heuristics.

**Version**: 0.1 (Feb 2020)
**Method**: Greedy allocation algorithm

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from f1ops.data_loader import get_european_races
from f1ops.geo import build_season_legs
from f1ops.cost import calculate_leg_cost
from f1ops.optimize import greedy_truck_allocation, optimize_fleet_allocation
from f1ops.config import DEFAULT_COST_PARAMS

%matplotlib inline
sns.set_style('whitegrid')

## 1. Problem Definition

**Fleet Allocation Problem**:
- F1 teams transport 40+ tonnes of equipment per race
- Fleet of trucks with limited capacity
- Must allocate loads to minimize costs

**Constraints**:
- Each truck has a capacity limit (typically 20-25 tonnes)
- All equipment must be transported
- Trucks travel together (convoy logistics)

**Objective**: Minimize total operational cost

## 2. Load Season Data

In [ ]:
races_2019 = get_european_races(2019)
legs = build_season_legs(races_2019)

print(f"Loaded {len(races_2019)} races, {len(legs)} legs")

## 3. Greedy Allocation Algorithm

**Algorithm**: First-Fit Decreasing (FFD)
1. Sort items by weight (descending)
2. For each item:
   - Try to fit in first available truck
   - If no truck has capacity, fail

**Properties**:
- Fast: O(n log n) complexity
- Simple to implement
- Approximation ratio: ≤ 11/9 OPT + 6/9

In [ ]:
# Define cargo items (example for one race)
cargo_items = [
    {"name": "Garage Equipment", "weight_kg": 8000},
    {"name": "Spare Parts", "weight_kg": 6000},
    {"name": "Tools & Machinery", "weight_kg": 5000},
    {"name": "Pit Equipment", "weight_kg": 4500},
    {"name": "Hospitality", "weight_kg": 7000},
    {"name": "Electronics", "weight_kg": 3000},
    {"name": "Tyres", "weight_kg": 4000},
    {"name": "Fuel & Fluids", "weight_kg": 2500},
]

total_cargo = sum(item["weight_kg"] for item in cargo_items)
print(f"Total cargo weight: {total_cargo:,} kg ({total_cargo/1000:.1f} tonnes)")

In [ ]:
# Fleet configuration
num_trucks = 8
truck_capacity_kg = 20000  # 20 tonnes per truck

print(f"Fleet configuration:")
print(f"  Number of trucks: {num_trucks}")
print(f"  Capacity per truck: {truck_capacity_kg:,} kg ({truck_capacity_kg/1000:.0f} tonnes)")
print(f"  Total fleet capacity: {num_trucks * truck_capacity_kg:,} kg ({num_trucks * truck_capacity_kg/1000:.0f} tonnes)")
print(f"\nUtilization: {total_cargo / (num_trucks * truck_capacity_kg) * 100:.1f}%")

In [ ]:
# Run greedy allocation
allocation = greedy_truck_allocation(cargo_items, num_trucks, truck_capacity_kg)

print("Allocation Results:\n")
for truck_id, items in allocation.items():
    truck_load = sum(item["weight_kg"] for item in items)
    utilization = truck_load / truck_capacity_kg * 100
    print(f"Truck {truck_id}:")
    print(f"  Load: {truck_load:>6,} kg ({utilization:5.1f}% capacity)")
    print(f"  Items: {', '.join(item['name'] for item in items)}")
    print()

## 4. Visualize Allocation

In [ ]:
# Prepare data for visualization
truck_loads = []
for truck_id in range(1, num_trucks + 1):
    items = allocation.get(truck_id, [])
    load = sum(item["weight_kg"] for item in items)
    truck_loads.append(load)

# Bar chart of truck loads
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(1, num_trucks + 1)
bars = ax.bar(x, truck_loads, color='#4a90e2', alpha=0.7, edgecolor='black', linewidth=1.5)
ax.axhline(truck_capacity_kg, color='red', linestyle='--', linewidth=2, label=f'Capacity: {truck_capacity_kg:,} kg')

# Add utilization percentages on bars
for i, (bar, load) in enumerate(zip(bars, truck_loads)):
    utilization = load / truck_capacity_kg * 100
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 500,
            f'{utilization:.1f}%',
            ha='center', va='bottom', fontweight='bold')

ax.set_xlabel('Truck Number', fontsize=12)
ax.set_ylabel('Load (kg)', fontsize=12)
ax.set_title('Greedy Allocation - Truck Loads', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Fleet Size Optimization

In [ ]:
# Test different fleet sizes
min_trucks = int(np.ceil(total_cargo / truck_capacity_kg))
max_trucks = 12

fleet_analysis = []

for n in range(min_trucks, max_trucks + 1):
    try:
        alloc = greedy_truck_allocation(cargo_items, n, truck_capacity_kg)
        
        # Calculate average utilization
        total_load = sum(sum(item["weight_kg"] for item in items) for items in alloc.values())
        avg_utilization = total_load / (n * truck_capacity_kg) * 100
        
        fleet_analysis.append({
            'num_trucks': n,
            'avg_utilization': avg_utilization,
            'feasible': True
        })
    except ValueError:
        fleet_analysis.append({
            'num_trucks': n,
            'avg_utilization': 0,
            'feasible': False
        })

df_fleet = pd.DataFrame(fleet_analysis)
df_fleet

In [ ]:
# Plot fleet size vs utilization
feasible_fleets = df_fleet[df_fleet['feasible']]

plt.figure(figsize=(10, 6))
plt.plot(feasible_fleets['num_trucks'], feasible_fleets['avg_utilization'], 
         marker='o', linewidth=2, markersize=10, color='#2ecc71')
plt.axhline(80, color='orange', linestyle='--', label='Target: 80% utilization')
plt.xlabel('Number of Trucks')
plt.ylabel('Average Utilization (%)')
plt.title('Fleet Size vs Average Utilization')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 6. Cost Impact of Fleet Size

In [ ]:
# Calculate cost for different fleet sizes using first leg as example
example_leg = legs[0]
params = DEFAULT_COST_PARAMS.copy()

cost_analysis = []

for n in feasible_fleets['num_trucks']:
    params['num_trucks'] = n
    cost = calculate_leg_cost(example_leg, params)
    cost_analysis.append({
        'num_trucks': n,
        'total_cost': cost.total_cost_eur,
        'cost_per_truck': cost.total_cost_eur / n
    })

df_cost = pd.DataFrame(cost_analysis)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(df_cost['num_trucks'], df_cost['total_cost'], marker='o', linewidth=2, markersize=8, color='#e74c3c')
ax1.set_xlabel('Number of Trucks')
ax1.set_ylabel('Total Cost (EUR)')
ax1.set_title('Total Cost vs Fleet Size')
ax1.grid(True, alpha=0.3)

ax2.plot(df_cost['num_trucks'], df_cost['cost_per_truck'], marker='s', linewidth=2, markersize=8, color='#9b59b6')
ax2.set_xlabel('Number of Trucks')
ax2.set_ylabel('Cost per Truck (EUR)')
ax2.set_title('Cost per Truck vs Fleet Size')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Example leg: {example_leg.leg_name}")
print(f"Distance: {example_leg.distance_km:.2f} km\n")
print(df_cost.to_string(index=False))

## 7. Limitations of Greedy Approach

**Advantages**:
- Fast execution (< 1ms for typical problems)
- Simple to understand and implement
- Reasonable solutions for most cases

**Limitations**:
- Not guaranteed to find optimal solution
- Can leave "gaps" in truck utilization
- Doesn't consider time windows or routing constraints
- No look-ahead or backtracking

**Future Improvements** (v0.4+):
- Google OR-Tools for optimal VRP solutions
- Capacitated Vehicle Routing Problem (CVRP)
- Time Windows (VRPTW)
- Multi-depot routing

## Summary

This notebook demonstrated:
- Fleet allocation problem formulation
- Greedy First-Fit Decreasing algorithm
- Visualization of truck load distribution
- Fleet size optimization analysis
- Cost impact of fleet sizing decisions

**Key Findings**:
- Minimum fleet size: 3 trucks (for 40 tonnes @ 20t capacity)
- Optimal fleet size: 3-4 trucks (balance of cost and utilization)
- Larger fleets increase total cost but provide redundancy
- Total cost scales linearly with fleet size

**Next**: Interactive visualization dashboard